In [1]:

#Objectifs:
#  Programmer sous R:
#    1) Une fonction pour la (TRAIN-TEST) validation répétée
#    2) Une fonction pour la validation croisée (K feuilles)
#    3) Une fonction pour la validation croisée imbriquée
#    4) Une fonction pour le Leave One Out (avec k = n)
# 
# deux critères à valider: moyenne des performances et écart-type des 

# construction de modèles (ou librairies de notre choix):
# library(rpart) = algos de décisions -> CART: classification and regression tree (classif ou regression)
# library(MASS) = algo de décisions -> LDA : linear decision analysis: (classif binaire)
# library(FNN) : K plus proche voisin (k-PPV)
# tree <- rpart(X10 ~ ., data= mesData, method= 'class')  ici ~ = en fonction de et . = toutes les autres variables
# tree <- rpart(X10 ~ ., data= mesData, method= 'class', control=rpart.control(minsplit=50), cp= val entre 0 et 1)  ici ~ = en fonction de et . = toutes les autres variables
# predict(tree, mesNouvellesDonnées, type='class')
# predict(tree, mesNouvellesDonnées, type='prob')
# JEUX de données madoc: Breiman: wave5000

# attente livrable: les fonctions, mini rapport explication fonction et comparaison des modèles(1,2,3,4) :robustesse sur les données de Breiman

# en sortie un vecteur mélanger de 1 à n: en entré un entier
# fonction sample déjà défini


# Question 1
#data = dataframe avec une colonne réel
#k = le nombre de répétition
#predictColumn = string = nom de la colonne X10
library(rpart)
validationRepeteeRPART = function (data, k) {
  errorRate = c();
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData$X10~.,modalData, method="class")
    prediction = predict(tree, testData, type="class")
    
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction)
    
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

library(MASS)

validationRepeteeLDA = function (data, k) {
  errorRate = c()
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = lda(modalData$X10~.,modalData, method="moment")
    prediction = predict(tree, testData, type="moment")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction$class)
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

#Exercice 2
#data = dataframe
#partitions = integer
validationCroiseeRPART = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData_step2$X10~.,modalData_step2, method="class")
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}




validationCroiseeLDA = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = lda(X10~.,modalData_step2)
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction$class)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}


#appel de fonction validationCroisee défini précédemment
#en paramètre k = le nombre de répétition
validationCroiseeRepetee = function (data, partitions, k) {
  robustesseMean = 0
  errorRateMean = 0
  
  for (i in 1:k) {
    robustesseMean = robustesseMean + validationCroiseeRPART(data, partitions, 1)
    errorRateMean = errorRateMean + validationCroiseeRPART(data, partitions, 0)
  }
  
  robustesseMean = robustesseMean/k
  errorRateMean = errorRateMean/k
  
  return(c(robustesseMean, errorRateMean))
  
}

#data = dataframe
#partitions = nombre de partition où on va split le dataset
validationCroiseeImpliquee = function(data, partitions) {
  
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  listErrorRate = c()
  
  for (i in 1:partitions) {
    
    errorRate = c()
    
    for (i in 1:partitions-1) {
 
      tempSetPartitions = setPartitions
      currentTestData = setPartitions[[i]]
      currentModalDataList = tempSetPartitions[-i]
      #fusion dataframe
      currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
      
      for (j in 3:(partitions-2)) {
        currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
      }
      
      numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
      numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
      
      testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
      modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
      
      #on calcule la précision (le taux d'erreurs)
      tree = lda(X10~.,modalData_step2)
      prediction = predict(tree, testData_step2, type="class")
      #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
      contingencyTable = table(testData_step2$X10, prediction$class)
      
      total = 0
      sumError = 0
      for (i in 1:nrow(contingencyTable)) {
        for (j in 1:ncol(contingencyTable)) {
          if (i != j) {
            sumError = sumError + contingencyTable[i,j]
          }
          total = total + contingencyTable[i,j]
        }
      }
      errorRate = c(errorRate, c(sumError/total))
    }
    
    listErrorRate = c(c(listErrorRate), mean(errorRate))
    
  }
  
  return(listErrorRate)
  
}

#appel de la fonction validationCroisee
#en paramètre data: un dataframe
leaveOneOut = function(data) {
  numberOfrows = nrow(data)
  return(c(validationCroiseeRPART(data, numberOfrows, 0), validationCroiseeRPART(data, numberOfrows, 1)))
}


#appel de tous les fonctions définies précédemment

algorithmSelection = function(data, K) {
  
  validationCroiseeRPARTErrorRate = validationRepeteeLDA(data, K)
  validationCroiseeLDAErrorRate = validationRepeteeRPART(data, K)
  
  if (validationCroiseeRPARTErrorRate > validationCroiseeLDAErrorRate) {
    return("SOLUTION: LDA")
  } else {
    return("SOLUTION: RPART")
  }
  
  return("ERROR")
}

validationCroiseeGetModel = function(data, partitions, colonneCible, errorRateThreshold = 0.3) {
  errorRate = c()
  #on m\u{fffd}lange les donn\u{fffd}es
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la pr\u{fffd}cision (le taux d'erreurs)
    #formulaString = paste("modalData_step2$", colonneCible, "~.",sep="")
    #formulaCible= as.formula(formulaString)
    #tree = rpart(formulaCible, modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
      
    tree = rpart(modalData_step2$X1~., modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
    prediction = predict(tree, testData_step2, type="class")
      
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    #formulaString = paste("testData_step2", colonneCible,sep="$")
    #formulaCible2 = as.formula(formulaString)
      
    contingencyTable = table(testData_step2$X1, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la m\u{fffd}me que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (mean(errorRate) <= errorRateThreshold) {
    listResult = list(mean(errorRate), tree)
    print("Error Rate < 0.3")
    print(mean(errorRate))
    return(listResult)
  }else{
    listResult = list(0, tree)
    print("Error Rate > 0.3, WRONG MODEL")
    print(mean(errorRate))
    return(listResult)
  }
}

#Variable cible : PREDICTION CHIFFRE D'AFFAIRES
GetModelChiffreAffaires = function(data, partitions){
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,9)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionChiffreAffaire = predict(listResult[[2]], donneesTest, type="class")
  print(predictionChiffreAffaire)
}

#Variable cible : CAPACITE D'EMPRUNT
GetModelCapaciteEmprunt = function(data, partitions){
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.6)
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionCapaciteEmprunt = predict(listResult[[2]], donneesTest, type="class")
  print(predictionCapaciteEmprunt)
}

GetModelCapaciteSecteurOne = function(data, partitions){
#Variable cible : SECTEUR 1
  result = 0
  while(result == 0){
    print("generating new model ...")
    donneesAppr = data[ , -c(1,3,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  print(listResult)
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionChiffreAffaire = predict(listResult[[2]], donneesTest, type="class")
  print(predictionChiffreAffaire)
}

In [2]:
apprentissage <- read.table("apprentissage.csv", sep = "," ,header = FALSE)
test <- read.table("test.csv", sep = "," ,header = FALSE)
GetModelChiffreAffaires(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.2333333
[[1]]
[1] 0.2333333

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 15 0 (0.4 0.12 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04)  
   2) X2< 0.5 10  0 0 (1 0 0 0 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 15 12 17500 (0 0.2 0.067 0.067 0.067 0.067 0.067 0.067 0.067 0.067 0.067 0.067 0.067 0.067)  
     6) X2< 1.5 6  3 17500 (0 0.5 0.17 0.17 0 0 0.17 0 0 0 0 0 0 0)  
      12) X6< 445.715 3  0 17500 (0 1 0 0 0 0 0 0 0 0 0 0 0 0) *
      13) X6>=445.715 3  2 20000 (0 0 0.33 0.33 0 0 0.33 0 0 0 0 0 0 0) *
     7) X2>=1.5 9  8 35000 (0 0 0 0 0.11 0.11 0 0.11 0.11 0.11 0.11 0.11 0.11 0.11)  
      14) X2< 2.5 2  1 35000 (0 0 0 0 0.5 0.5 0 0 0 0 0 0 0 0) *
      15) X2>=2.5 7  6 52500 (0 0 0 0 0 0 0 0.14 0.14 0.14 0.14 0.14 0.14 0.14)  
        30) X2< 4 2  1 52500 (0 0 0 0 0 0 0 0.5 0.5 0 0 0 0 0) *
        31) X2>=4 5  4 101000 (0 0 0 0 0 0 0 0 0 0.2 0.2 0.2 0.2 0.2)  
          62) X2< 7 2  1 1010

In [3]:
GetModelCapaciteEmprunt(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.85
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "generating new model ..."


...

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.7
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6833333
[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] "Error Rate < 0.3"
[1] 0.5333333
[[1]]
[1] 0.5333333

[[2]]
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 14 10000 (0.04 0.24 0.04 0.44 0.04 0.04 0.04 0.04 0.04 0.04)  
   2) X28< 56.425 14  4 10000 (0 0.14 0.071 0.71 0 0.071 0 0 0 0)  
     4) X23< 814.595 12  2 10000 (0 0.083 0.083 0.83 0 0 0 0 0 0)  
       8) X26< 1.58 5  2 10000 (0 0.2 0.2 0.6 0 0 0 0 0 0)  
        16) X2< 0.5 2  1 5000 (0 0.5 0.5 0 0 0 0 0 0 0) *
        17) X2>=0.5 3  0 10000 (0 0 0 1 0 0 0 0 0 0) *
       9) X26>=1.58 7  0 10000 (0 0 0 1 0 0 0 0 0 0) *
     5) X23>=814.595 2  1 5000 (0 0.5 0 0 0 0.5 0 0 0 0) *
   3) X28>=56.425 11  7 5000 (0.091 0.36 0 0.091 0.091 0 0.091 0.091 0.091 0.091)  
     6) X10< 40.3 5  1 5000 (0 0.8 0 0.2 0 0 0 0 0 0) *
     7) X10>=40.3 6  5 3000 (0.17 0 0 0 0.17 0 0.17 0.17 0.17 0.17)  
      14) X2< 6.5 3  2 3000 (0.33 0 0 0 0.33 0 0 0.33 0 0) *
      15) X2>=6.5 3  2 30000 (0 0 0 0 0 0 0.33 0 0.33 0.33) *

    1     2     3     4    

In [5]:
GetModelCapaciteSecteurOne(apprentissage, 10)

[1] "generating new model ..."


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

ERROR: Error in cbind(yval2, yprob, nodeprob): le nombre de lignes des matrices doit correspondre (voir argument 2)
